In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

from IPython.display import Markdown

In [2]:
TITLE_SIZE = 18
HIST_BINS = 51

rng = np.random.default_rng(42)

## 1 Simulation 

In [3]:
from collections import deque

# lambda = mean_arrival_rate
# mu = mean_service_rate
def run_sim_and_plot(n, mean_arrival_rate, mean_service_rate, buffer=None, service_dist=None):
    params = build_params(n, mean_arrival_rate, mean_service_rate, buffer, service_dist)
#     display(Markdown(
#         f"### Simulation: n={n}, " +
#         f"$\lambda$={format(params['mean_arrival_rate'])}, " +
#         f"$\mu$={format(params['mean_service_rate'])}, " +
#         f"$1/\lambda$={format(params['mean_interarrival_time'])}, " +
#         f"$1/\mu$={format(params['mean_service_time'])}, " +
#         f"$Buffer$={format(params['buffer'])}, " + 
#         f"$Distribution$={service_dist}"
#     ))
    result = run_sim(params)
#     dump_stats(result)
#     plot_result(result)
    return result

In [4]:
def build_params(num_jobs, mean_arrival_rate, mean_service_rate, buffer, service_dist):
    dist_dict = {
        "exp": lambda n : rng.exponential(scale=1/mean_service_rate, size=n),
        "log_normal": lambda n : rng.lognormal(mean=(-3/2)*np.log(mean_service_rate), sigma=np.log(mean_service_rate)**0.5, size=n),
        "uniform" : lambda n : rng.uniform(low=0.5/mean_service_rate, high=1.5/mean_service_rate, size=n),
    }
    return {
        "n": num_jobs,
        "mean_arrival_rate": mean_arrival_rate,
        "mean_service_rate": mean_service_rate,
        "mean_interarrival_time": 1.0 / mean_arrival_rate,
        "mean_service_time": 1.0 / mean_service_rate,
        "num_bins": int(num_jobs / mean_arrival_rate),
        "buffer" : buffer-1 if buffer is not None else np.inf,
        "service_dist" : dist_dict["exp" if service_dist is None else service_dist]
    }

In [5]:
def run_sim(params):
    n = params["n"]
    buffer = params["buffer"]
    
    # Parameters
    mean_interarrival_time = params["mean_interarrival_time"]
    mean_service_time = params["mean_service_time"]
    
    # Simulation data and results
    interarrival_times = rng.exponential(scale=mean_interarrival_time, size=n)
    arrival_times = np.cumsum(interarrival_times)
    service_times = params["service_dist"](n)
    
    curr_time = 0
    no_of_jobs_in_system = 0
    jobs_df = pd.DataFrame({
        "interarrival_time": interarrival_times,
        "arrive_time": arrival_times,
        "service_time": service_times,
        "start_time": np.zeros(n),
        "depart_time": np.zeros(n)
    })
    
    jobs_df.loc[0, "start_time"] = jobs_df.loc[0, "arrive_time"]
    jobs_df.loc[0, "depart_time"] = jobs_df.loc[0, "start_time"] + jobs_df.loc[0, "service_time"]
    q = deque()
    recent_job = {
        'depart_time' : -1,
        'start_time' : -1
    }
    
    def process_queue(curr_time):
        nonlocal recent_job
        # pre processing of queue
        while len(q) > 0:
            curr_job = q[0]
            i = curr_job['index']
            # we log the start time of the jobs
            dt = recent_job['start_time'] + jobs_df.loc[i, "service_time"]
            if (dt <= curr_time):
                # if job has been completed, mark it complete in the jobs df
                q.popleft()
                jobs_df.loc[i, "start_time"] = recent_job['start_time']
                jobs_df.loc[i, "depart_time"] = dt
                jobs_df.loc[i, "response_time"] = jobs_df.loc[i, "depart_time"] - jobs_df.loc[i, "arrive_time"]
                jobs_df.loc[i, "wait_time"] = jobs_df.loc[i, "start_time"] - jobs_df.loc[i, "arrive_time"]
                recent_job = {
                    'depart_time' : dt,
                    'start_time' : dt
                }
            else:
                break
    
    for i2 in range(1, n+1):
        curr_time = np.inf if i2==n else jobs_df.loc[i2, "arrive_time"]
        process_queue(curr_time)
        
        if i2==n:
            break
        
        jobs_in_queue = len(q)
        jobs_in_system = 0
        if len(q) > 0:
            jobs_in_queue -= 1
            jobs_in_system = 1
        
        # we need to drop the job
        if jobs_in_system == 1 and jobs_in_queue == buffer:
            jobs_df.loc[i2, "start_time"] = np.nan
            jobs_df.loc[i2, "depart_time"] = np.nan
        
        else:
            if jobs_in_system == 0:
                # if queue is empty, job will start as soon as it arrives
                recent_job = {
                    'depart_time' : -1,
                    'start_time' : jobs_df.loc[i2, 'arrive_time']
                }
            q.append({
                'index' : i2,
            })
    
#     assert(len(q)==0)
    missing_rows = (jobs_df.isna().sum(axis=1) > 0).sum()
    jobs_df = jobs_df.dropna()
    jobs_df = jobs_df.reset_index(drop=True)
#     print(jobs_df)
#     assert((jobs_df>=0).all(axis=None))
#     print(jobs_df)
#     events_df = build_events_df(params, jobs_df)
#     print(events_df)
#     total_width = get_total_width(jobs_df)
    
#     sim_mean_interarrival_time = jobs_df["interarrival_time"].mean()
#     sim_mean_arrival_rate = 1.0 / sim_mean_interarrival_time
#     sim_mean_service_time = jobs_df["service_time"].mean()
#     sim_mean_service_rate = 1.0 / sim_mean_service_time
    sim_mean_wait_time = jobs_df["wait_time"].mean()
#     sim_response_time_mean = jobs_df["response_time"].mean()
#     sim_response_time_var = jobs_df["response_time"].var()
    
    # mean_num_jobs_in_system and mean_num_jobs_in_queue
#     width = events_df["width"]
#     total_weighted_num_jobs_in_system = (width * events_df["num_jobs_in_system"]).sum()
#     total_weighted_num_jobs_in_queue = (width * events_df["num_jobs_in_queue"]).sum()
#     sim_mean_num_jobs_in_system = total_weighted_num_jobs_in_system / total_width
#     sim_mean_num_jobs_in_queue = total_weighted_num_jobs_in_queue / total_width
    
#     # throughput mean and variance
#     departures = events_df.loc[events_df["num_jobs_in_system_change"] == -1.0, "lo_bd"]
#     hist, _ = np.histogram(departures, bins=int(total_width) + 1)
#     sim_throughput_mean = np.mean(hist)
    
#     # utilization
#     util = estimate_util(jobs_df)
    
    return {
#         "params": params,
#         "jobs_df": jobs_df,
#         "events_df": events_df,
#         "total_duration": total_width,
#         "mean_arrival_rate": sim_mean_arrival_rate,
#         "mean_interarrival_time": sim_mean_interarrival_time,
#         "mean_service_rate": sim_mean_service_rate,
#         "mean_service_time": sim_mean_service_time,
        "mean_wait_time": sim_mean_wait_time,
#         "response_time_mean": sim_response_time_mean,
#         "response_time_var": sim_response_time_var,
#         "mean_num_jobs_in_system": sim_mean_num_jobs_in_system,
#         "mean_num_jobs_in_queue": sim_mean_num_jobs_in_queue,
#         "throughput_mean": sim_throughput_mean,
#         "utilization": util,
        "blocked_jobs" : missing_rows
    }

In [6]:
# Serialize the jobs into events (arrival, start, departure) so we can compute job counts.
def build_events_df(params, jobs_df):
    n = jobs_df.shape[0]
    arrivals = jobs_df["arrive_time"]
    starts = jobs_df["start_time"]
    departures = jobs_df["depart_time"]
    
    # width = up_bd - lo_bd, num_jobs_in_queue = num_jobs_in_system - 1
    events_df = pd.DataFrame(columns=["lo_bd", "up_bd", "width", "num_jobs_in_system", "num_jobs_in_queue"])

    lo_bd = 0.0
    arrive_idx = 0
    start_idx = 0
    depart_idx = 0
    num_jobs_in_system = 0
    num_jobs_in_queue = 0
    
    while depart_idx < n:
        arrival = arrivals[arrive_idx] if arrive_idx < n else float("inf")
        start = starts[start_idx] if start_idx < n else float("inf")
        departure = departures[depart_idx]
                        
        if arrival <= start and arrival <= departure:
            up_bd = arrival
            n_change, nq_change = 1, 1
            arrive_idx = arrive_idx + 1        
        elif start <= arrival and start <= departure:
            up_bd = start
            n_change, nq_change = 0, -1
            start_idx = start_idx + 1
        else:
            up_bd = departure
            n_change, nq_change = -1, 0
            depart_idx = depart_idx + 1
                
        width = up_bd - lo_bd
        events_df = events_df.append({
            "lo_bd": lo_bd,
            "up_bd": up_bd,
            "width": width,
            "num_jobs_in_system": num_jobs_in_system,
            "num_jobs_in_queue": num_jobs_in_queue,
            "num_jobs_in_system_change": n_change,
            "num_jobs_in_queue_change": nq_change,
        }, ignore_index=True)
        
        num_jobs_in_system = num_jobs_in_system + n_change
        num_jobs_in_queue = num_jobs_in_queue + nq_change

        lo_bd = up_bd
    
    return events_df

def get_total_width(jobs_df):
    return jobs_df.iloc[-1]["depart_time"] - jobs_df.iloc[0]["arrive_time"]

def estimate_util(jobs_df):
    busy = (jobs_df["depart_time"] - jobs_df["start_time"]).sum()
    return busy / get_total_width(jobs_df)

In [7]:
# STATS DUMPS

def format(value):
    return f"{value:,.4f}"

def dump_stats(result):
    params = result["params"]
    jobs_df = result["jobs_df"]
    response_time = jobs_df["response_time"]
    arrival_rate_mean = result["mean_arrival_rate"]
    service_rate_mean = result["mean_service_rate"]
    service_time_mean = result["mean_service_time"]
    response_time_mean = result["response_time_mean"]
    throughput_mean = result["throughput_mean"]
    util = result["utilization"]
    num_jobs_in_system_mean = result["mean_num_jobs_in_system"]
    
    print("Simulation statistics")
    print("---------------------")
    print("total_duration          = " + format(result['total_duration']))
    print("arrival_rate_mean       = " + format(arrival_rate_mean))
    print("interarrival_time_mean  = " + format(result['mean_interarrival_time']))
    # Note: E[response_time] = E[wait_time] + E[service_time]
    print("response_time")
    print("  mean                  = " + format(response_time_mean))
    print("  var                   = " + format(result['response_time_var']))
    print("  p50                   = " + format(np.quantile(response_time, .5)))
    print("  p75                   = " + format(np.quantile(response_time, .75)))
    print("  p90                   = " + format(np.quantile(response_time, .90)))
    print("  p95                   = " + format(np.quantile(response_time, .95)))
    print("  p99                   = " + format(np.quantile(response_time, .99)))
    print("wait_time_mean          = " + format(result['mean_wait_time']))
    print("service_rate_mean       = " + format(service_rate_mean))
    print("service_time_mean       = " + format(service_time_mean))
    print("num_jobs_in_system_mean = " + format(num_jobs_in_system_mean))
    print("num_jobs_in_queue_mean  = " + format(result['mean_num_jobs_in_queue']))
    print("throughput_mean         = " + format(throughput_mean))
    print("utilization             = " + format(util))
    print("")
    print("Little's Law: E[N] = lambda * E[T]")
    print("----------------------------------")
    print("num_jobs_in_system_mean                = " + format(num_jobs_in_system_mean))
    print("arrival_rate_mean * response_time_mean = " + format(arrival_rate_mean * response_time_mean) +
          " (= " + format(arrival_rate_mean) + " * " + format(response_time_mean) + ")")
    print("")
    print("Utilization Law, version 1: rho_i = lambda_i / mu_i")
    print("---------------------------------------------------")
    print("utilization                            = " + format(util))
    print("arrival_rate_mean                      = " + format(arrival_rate_mean))
    print("service_rate_mean                      = " + format(service_rate_mean))
    print("arrival_rate_mean / service_rate_mean  = " + format(arrival_rate_mean / service_rate_mean) +
          " (= " + format(arrival_rate_mean) + " / " + format(service_rate_mean) + ")")
    print("")
    print("Utilization Law, version 2: rho_i = X_i * E[S]")
    print("----------------------------------------------")
    print("utilization                            = " + format(util))
    print("throughput_mean                        = " + format(throughput_mean))
    print("service_time_mean                      = " + format(service_time_mean))
    print("throughput_mean * service_time_mean    = " + format(throughput_mean * service_time_mean) +
          " (= " + format(throughput_mean) + " * " + format(service_time_mean) + ")")
    print("")
    print("Blocked jobs")
    print("----------------------------------------------")
    print("No of jobs dropped                     = " + format(result["blocked_jobs"]))

In [8]:
# PLOTTING FUNCTIONS

def plot_result(result):
    params = result["params"]
    jobs_df = result["jobs_df"]
    events_df = result["events_df"]
    
#     _plot_jobs_gantt(params, jobs_df)
#     _plot_jobs_over_time(events_df)
#     _plot_histogram(params, jobs_df["interarrival_time"], "Histogram of interarrival times", "Interarrival time")
#     _plot_histogram(params, jobs_df["arrive_time"], "Histogram of arrival times", "Arrival time")
#     _plot_histogram(params, jobs_df["wait_time"], "Histogram of wait times", "Wait time")
#     _plot_histogram(params, jobs_df["service_time"], "Histogram of service times", "Service time")
#     _plot_histogram(params, jobs_df["response_time"], "Histogram of response times", "Response time")

def _plot_histogram(params, data, title, xlabel):
    plt.figure(figsize=(14, 2))
    plt.title(title, size=TITLE_SIZE)
    plt.xlabel(xlabel)
    plt.ylabel("Count")
    plt.hist(data, bins=HIST_BINS)
    plt.show()

def _plot_jobs_gantt(params, jobs_df):
    n = params["n"]
    start_job = int(n / 2)
    end_job = start_job + 40
    trunc_df = jobs_df[start_job:end_job]
    
    plt.figure(figsize=(14, 8))
    plt.title("Job schedule (partial view)", size=TITLE_SIZE)
    plt.xlabel("Time")
    plt.ylabel("Job ID")
    plt.barh(
        y=trunc_df.index,
        left=trunc_df["arrive_time"],
        width=trunc_df["response_time"],
        alpha=1.0,
        color="gainsboro")
    plt.barh(
        y=trunc_df.index,
        left=trunc_df["start_time"],
        width=trunc_df["service_time"],
        alpha=1.0,
        color="limegreen")
    plt.gca().invert_yaxis()
    plt.grid(axis="x")
    plt.show()

# FIXME Departures shouldn't count here.
def _plot_jobs_over_time(events_df):
    plt.figure(figsize=(14, 2))
    plt.title("# jobs in system over time", size=TITLE_SIZE)
    plt.xlabel("Time")
    plt.ylabel("Job count")
    print(events_df[["lo_bd", "num_jobs_in_system"]])
    plt.plot(events_df["lo_bd"], events_df["num_jobs_in_system"])
    plt.show()

In [9]:
results_expt_1 = {}
results_expt_2 = {}

In [11]:
# run 1000 expt
# change lambda
for l in [8, 16, 24, 32, 50, 64]:
    for service_dist in ["exp", "log_normal", "uniform"]:
        mu = 64
        rho = l / 64
        no_of_expts = 10
        avg_metrics = {"mwt" : 0, "bj" : 0}
        for _ in range(no_of_expts):
            result = run_sim_and_plot(2000, l, mu, 5, service_dist)
            print("done")
            mwt = result["mean_wait_time"]
            bj = result["blocked_jobs"] / 2000
            avg_metrics["mwt"] += mwt
            avg_metrics["bj"] += bj
        print()
        avg_metrics["mwt"] /= no_of_expts
        avg_metrics["bj"] /= no_of_expts
        avg_metrics["rho"] = rho
        results_expt_1[(l, service_dist)] = avg_metrics

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done



In [12]:
for b in [1, 2, 5, 10, 15, 20]:
    for service_dist in ["exp", "log_normal", "uniform"]:
        l = 16
        mu = 64
        rho = l / 64
        no_of_expts = 10
        avg_metrics = {"mwt" : 0, "bj" : 0}
        for _ in range(no_of_expts):
            result = run_sim_and_plot(2000, l, mu, b, service_dist)
            print("done")
            mwt = result["mean_wait_time"]
            bj = result["blocked_jobs"] / 2000
            avg_metrics["mwt"] += mwt
            avg_metrics["bj"] += bj
        print()
        avg_metrics["mwt"] /= no_of_expts
        avg_metrics["bj"] /= no_of_expts
        avg_metrics["rho"] = rho
        results_expt_2[(b, service_dist)] = avg_metrics

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done

done
done
done
done
done
done
done
done
done
done



In [ ]:
results_expt_2

In [13]:
import pandas as pd

In [14]:
df1 = pd.DataFrame.from_dict(results_expt_1).T

In [15]:
df1.to_csv("expt1.csv")

In [16]:
df1

mwt       bj      rho
8  exp         0.002276  0.00055  0.12500
   log_normal  0.017236  0.01945  0.12500
   uniform     0.001224  0.00050  0.12500
16 exp         0.005014  0.00100  0.25000
   log_normal  0.022571  0.05860  0.25000
   uniform     0.002845  0.00055  0.25000
24 exp         0.008402  0.00515  0.37500
   log_normal  0.031888  0.11445  0.37500
   uniform     0.005109  0.00110  0.37500
32 exp         0.012676  0.01390  0.50000
   log_normal  0.032623  0.15100  0.50000
   uniform     0.008020  0.00270  0.50000
50 exp         0.023705  0.08620  0.78125
   log_normal  0.036504  0.22660  0.78125
   uniform     0.018671  0.03370  0.78125
64 exp         0.031356  0.16850  1.00000
   log_normal  0.038736  0.31880  1.00000
   uniform     0.029403  0.10910  1.00000

In [17]:
df1 = df1.reset_index()

In [18]:
df1.pivot(index="rho", columns=["level_1"], values=["mwt", "bj"]).to_csv("expt1_pivot.csv")

In [19]:
df2 = pd.DataFrame.from_dict(results_expt_2).T

In [20]:
df2

mwt       bj   rho
1  exp         0.000000  0.19860  0.25
   log_normal  0.000000  0.18615  0.25
   uniform     0.000000  0.20295  0.25
2  exp         0.003051  0.04840  0.25
   log_normal  0.008056  0.11395  0.25
   uniform     0.001906  0.03025  0.25
5  exp         0.005144  0.00120  0.25
   log_normal  0.025154  0.06560  0.25
   uniform     0.002829  0.00050  0.25
10 exp         0.005163  0.00050  0.25
   log_normal  0.040758  0.03315  0.25
   uniform     0.002712  0.00050  0.25
15 exp         0.004810  0.00050  0.25
   log_normal  0.048022  0.01975  0.25
   uniform     0.002711  0.00050  0.25
20 exp         0.005153  0.00050  0.25
   log_normal  0.057970  0.01955  0.25
   uniform     0.002891  0.00050  0.25

In [21]:
df2.to_csv("expt2.csv")

In [22]:
df2 = df2.reset_index() 

In [23]:
df2

,level_0,level_1,mwt,bj,rho
0,1,exp,0.000000,0.19860,0.25
1,1,log_normal,0.000000,0.18615,0.25
2,1,uniform,0.000000,0.20295,0.25
3,2,exp,0.003051,0.04840,0.25
4,2,log_normal,0.008056,0.11395,0.25
5,2,uniform,0.001906,0.03025,0.25
6,5,exp,0.005144,0.00120,0.25
7,5,log_normal,0.025154,0.06560,0.25
8,5,uniform,0.002829,0.00050,0.25
9,10,exp,0.005163,0.00050,0.25


In [24]:
df2.pivot(index="level_0", columns=["level_1"], values=["mwt", "bj"]).to_csv("expt2_pivot.csv")